In [ ]:
import pandas as pd
ven = pd.read_csv("C:/Users/shubh/Documents/vendor_yes.csv")

def displayMinVendor(ven):
    ven["Unfulfilled_len"] = ven['Unfulfilled'].apply(lambda x: len(str(x).split(';')) if pd.notna(x) else 0)

    ven['Cost' + '_normalized'] = normalize_by_sum(ven['Cost'])
    ven['Days' + '_normalized'] = normalize_by_sum(ven['Days'])
    ven['Unfulfilled_len' + '_normalized'] = normalize_by_sum(ven['Unfulfilled_len'])

    
    ven['Overall'] = ven.apply(lambda row: 0.4 * row['Cost_normalized'] +
                                       0.35 * row['Days_normalized'] +
                                       0.25 * row['Unfulfilled_len_normalized'], axis=1)
    
    
    matching_row = ven[(ven['Cost'] == ven['Cost'].min()) & (ven['Days'] == ven['Days'].min())]

    if not matching_row.empty:
    # If a matching row exists, print the row
        print("Matching Row:")
        print(matching_row)
    else:
    # If no matching row exists, print the minimum values
        min_values_Cost = ven['Cost'].min()
        min_values_Days = ven['Days'].min()
        print(f"Minimum Cost Offered: {min_values_Cost}, Mimimum Days Offered: {min_values_Days}")
    
        min_values = ven['Overall'].nsmallest(1)
        result_rows = ven[ven['Overall'].isin(min_values)]

    return result_rows[['VendorID', 'Cost', 'Days', 'Unfulfilled']]
    
def normalize_by_sum(column):
    normalized_column = column / column.sum()
    return normalized_column

In [ ]:
displayMinVendor(ven)